#### **BEIR BM25**

This notebook uses the open-source elastic search BM25 to establish a baseline on BEIR

Author: Niklas Muennighoff

##### Download Elastic Search

In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
import time
time.sleep(20)

In [ ]:
# wait a bit then test - should show something like cluster_name" : "elasticsearch" ...
!curl -X GET "localhost:9200/"

{
  "name" : "fc060de88c38",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "6xWl5Pn4RtGiI9rl-Vo2FA",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


##### Run on BEIR - Sample

In [ ]:
!pip install -q beir

In [ ]:
from beir import util, LoggingHandler
import logging
# Code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
dataset = "trec-covid"

In [ ]:
from beir.datasets.data_loader import GenericDataLoader

url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = os.path.join(os.getcwd(), "datasets")
data_path = util.download_and_unzip(url, out_dir)
print("Dataset downloaded here: {}".format(data_path))

# Load the dataset into BEIR
data_path = f"datasets/{dataset}"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")

2021-11-23 03:57:01 - Downloading trec-covid.zip ...


/content/datasets/trec-covid.zip:   0%|          | 0.00/70.5M [00:00<?, ?iB/s]

2021-11-23 03:57:16 - Unzipping trec-covid.zip ...
Dataset downloaded here: /content/datasets/trec-covid
2021-11-23 03:57:18 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2021-11-23 03:57:20 - Loaded 171332 TEST Documents.
2021-11-23 03:57:20 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

#### Provide parameters for elastic-search
hostname = "localhost"
index_name = dataset
initialize = True # True, will delete existing index with same name and reindex all documents

model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)

#### Retrieve dense results (format of results is identical to qrels)
results = retriever.retrieve(corpus, queries)

2021-11-23 03:57:21 - Loading faiss with AVX2 support.
2021-11-23 03:57:21 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2021-11-23 03:57:21 - Loading faiss.
2021-11-23 03:57:21 - Successfully loaded faiss.
2021-11-23 03:57:21 - Activating Elasticsearch....
2021-11-23 03:57:21 - Elastic Search Credentials: {'hostname': 'localhost', 'index_name': 'trec-covid', 'keys': {'title': 'title', 'body': 'txt'}, 'timeout': 100, 'retry_on_timeout': True, 'maxsize': 24, 'number_of_shards': 'default', 'language': 'english'}
2021-11-23 03:57:21 - Deleting previous Elasticsearch-Index named - trec-covid
2021-11-23 03:57:21 - Creating fresh Elasticsearch-Index named - trec-covid


que: 100%|██████████| 1/1 [00:05<00:00,  5.94s/it]


In [ ]:
import json
# Save scores for top 1000 docs for each query, i.e. 1000 * queries lines
with open(f"./results_{dataset}.json", 'w') as fp:
    json.dump(results, fp)

In [ ]:
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - NDCG@1: 0.8200
2021-11-23 03:58:12 - NDCG@3: 0.7623
2021-11-23 03:58:12 - NDCG@5: 0.7249
2021-11-23 03:58:12 - NDCG@10: 0.6880
2021-11-23 03:58:12 - NDCG@100: 0.4845
2021-11-23 03:58:12 - NDCG@1000: 0.4330
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - MAP@1: 0.0023
2021-11-23 03:58:12 - MAP@3: 0.0062
2021-11-23 03:58:12 - MAP@5: 0.0096
2021-11-23 03:58:12 - MAP@10: 0.0170
2021-11-23 03:58:12 - MAP@100: 0.0852
2021-11-23 03:58:12 - MAP@1000: 0.2084
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - Recall@1: 0.0023
2021-11-23 03:58:12 - Recall@3: 0.0064
2021-11-23 03:58:12 - Recall@5: 0.0103
2021-11-23 03:58:12 - Recall@10: 0.0191
2021-11-23 03:58:12 - Recall@100: 0.1173
2021-11-23 03:58:12 - Recall@1000: 0.4128
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - P@1: 0.8800
2021-11-23 03:58:12 - P@3: 0.8000
2021-11-23 03:58:12 - P@5: 0.7680
2021-11-23 03:58:12 - P@10: 0.7340
2021-11-23 03:58:12 - P@100: 0.5084
2021-11-23 03:58:12 - P@1000: 0.2019


In [ ]:
with open(f"./results_{dataset}.json", 'r') as fp:
    results_loaded = json.load(fp)

In [ ]:
# Just evaluate
ndcg, _map, recall, precision = EvaluateRetrieval().evaluate(qrels, results, retriever.k_values)

2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - NDCG@1: 0.8200
2021-11-23 03:58:12 - NDCG@3: 0.7623
2021-11-23 03:58:12 - NDCG@5: 0.7249
2021-11-23 03:58:12 - NDCG@10: 0.6880
2021-11-23 03:58:12 - NDCG@100: 0.4845
2021-11-23 03:58:12 - NDCG@1000: 0.4330
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - MAP@1: 0.0023
2021-11-23 03:58:12 - MAP@3: 0.0062
2021-11-23 03:58:12 - MAP@5: 0.0096
2021-11-23 03:58:12 - MAP@10: 0.0170
2021-11-23 03:58:12 - MAP@100: 0.0852
2021-11-23 03:58:12 - MAP@1000: 0.2084
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - Recall@1: 0.0023
2021-11-23 03:58:12 - Recall@3: 0.0064
2021-11-23 03:58:12 - Recall@5: 0.0103
2021-11-23 03:58:12 - Recall@10: 0.0191
2021-11-23 03:58:12 - Recall@100: 0.1173
2021-11-23 03:58:12 - Recall@1000: 0.4128
2021-11-23 03:58:12 - 

2021-11-23 03:58:12 - P@1: 0.8800
2021-11-23 03:58:12 - P@3: 0.8000
2021-11-23 03:58:12 - P@5: 0.7680
2021-11-23 03:58:12 - P@10: 0.7340
2021-11-23 03:58:12 - P@100: 0.5084
2021-11-23 03:58:12 - P@1000: 0.2019


##### Run on BEIR - Base datasets

In [ ]:
import json

from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval

datasets_all = ["msmarco", "msmarco-v2", "trec-covid", "nfcorpus", "bioasq", "nq", "hotpotqa", "fiqa", 
                "signal1m", "trec-news", "arguana", "webis-touche2020", "cqadupstack", "quora", "dbpedia-entity",
                "scidocs", "fever", "climate-fever", "scifact", "robust04"]

# Subselecting all available ones
datasets = ["msmarco", "trec-covid", "nfcorpus", "nq", "hotpotqa", "fiqa", 
           "arguana", "webis-touche2020", "cqadupstack", "quora", "dbpedia-entity",
           "scidocs", "fever", "climate-fever", "scifact"]

#### Provide parameters for elastic-search
hostname = "localhost"
initialize = True # True, will delete existing index with same name and reindex all documents

ndcgs = {}

for i, dataset in enumerate(datasets):

    logging.info(f"\n{'-' * 10} Running {dataset} {'-' * 10}\n")

    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
    out_dir = os.path.join(os.getcwd(), "datasets")
    data_path = util.download_and_unzip(url, out_dir)
    print("Dataset downloaded here: {}".format(data_path))
    # Load the dataset into BEIR
    data_path = f"datasets/{dataset}"
    # In the paper it says, BEIR used the dev set for msmarco
    split = "dev" if dataset == "msmarco" else "test"

    # cqadupstack - Contains several sub datasets
    if dataset == "cqadupstack":
        cqa_ndcgs, cqa_maps, cqa_recalls, cqa_precisions = [], [], [], []
        for sub_dataset in os.listdir(data_path):
            sub_data_path = f"datasets/{dataset}/{sub_dataset}"
            corpus, queries, qrels = GenericDataLoader(sub_data_path).load(split=split)
            model = BM25(index_name=sub_dataset, hostname=hostname, initialize=initialize)
            retriever = EvaluateRetrieval(model)
            #### Retrieve dense results (format of results is identical to qrels)
            results = retriever.retrieve(corpus, queries)
            # Save scores for top 1000 docs for each query, i.e. 1000 * queries lines
            with open(f"./results_{dataset}_{sub_dataset}.json", 'w') as fp:
                json.dump(results, fp)

            ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

            cqa_ndcgs.append(ndcg)
            cqa_maps.append(_map)
            cqa_recalls.append(recall)
            cqa_precisions.append(precision)

        for (metric, group) in [(ndcg, cqa_ndcgs), (_map, cqa_maps), (recall, cqa_recalls), (precision, cqa_precisions)]:
            for k in metric.keys():
                metric[k] = sum([score[k] for score in group]) / len(group)

        logging.info("CQA Final")
        logging.info(f"{ndcg}")
        logging.info(f"{_map}")
        logging.info(f"{recall}")
        logging.info(f"{precision}")

    else:
        corpus, queries, qrels = GenericDataLoader(data_path).load(split=split)
        model = BM25(index_name=dataset, hostname=hostname, initialize=initialize)
        retriever = EvaluateRetrieval(model)
        #### Retrieve dense results (format of results is identical to qrels)
        results = retriever.retrieve(corpus, queries)
        # Save scores for top 1000 docs for each query, i.e. 1000 * queries lines
        with open(f"./results_{dataset}.json", 'w') as fp:
            json.dump(results, fp)
        ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)

    ndcgs[dataset] = ndcg
    # Clean-up each time to avoid running out of space
    !rm -r datasets

with open("./beir_bm25_ndcgs.json", 'w') as fp:
    json.dump(ndcgs, fp)

2021-11-10 06:13:58 - Loading faiss with AVX2 support.
2021-11-10 06:13:58 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2021-11-10 06:13:58 - Loading faiss.
2021-11-10 06:13:58 - Successfully loaded faiss.
2021-11-10 06:13:58 - 
---------- Running msmarco ----------

2021-11-10 06:13:58 - Downloading msmarco.zip ...


/content/datasets/msmarco.zip:   0%|          | 0.00/1.01G [00:00<?, ?iB/s]

2021-11-10 06:14:20 - Unzipping msmarco.zip ...
Dataset downloaded here: /content/datasets/msmarco
2021-11-10 06:14:50 - Loading Corpus...


  0%|          | 0/8841823 [00:00<?, ?it/s]

2021-11-10 06:15:51 - Loaded 8841823 DEV Documents.
2021-11-10 06:15:51 - Doc Example: {'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.', 'title': ''}
2021-11-10 06:15:51 - Loading Queries...
2021-11-10 06:15:53 - Loaded 6980 DEV Queries.
2021-11-10 06:15:53 - Query Example: how many years did william bradford serve as governor of plymouth colony?
2021-11-10 06:15:53 - Activating Elasticsearch....
2021-11-10 06:15:53 - Elastic Search Credentials: {'hostname': 'localhost', 'index_name': 'msmarco', 'keys': {'title': 'title', 'body': 'txt'}, 'timeout': 100, 'retry_on_timeout': True, 'maxsize': 24, 'number_of_shards': 'default', 'language': 'english'}
2021-11-10 06:15:53 - Deleting previous Elasticsearch-Ind

...

2021-11-10 08:41:46 - Unzipping scifact.zip ...
Dataset downloaded here: /content/datasets/scifact
2021-11-10 08:41:46 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

2021-11-10 08:41:47 - Loaded 5183 TEST Documents.
2021-11-10 08:41:47 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

que: 100%|██████████| 3/3 [00:06<00:00,  2.18s/it]


2021-11-10 08:42:00 - 

2021-11-10 08:42:00 - NDCG@1: 0.5100
2021-11-10 08:42:00 - NDCG@3: 0.5607
2021-11-10 08:42:00 - NDCG@5: 0.5895
2021-11-10 08:42:00 - NDCG@10: 0.6110
2021-11-10 08:42:00 - NDCG@100: 0.6329
2021-11-10 08:42:00 - NDCG@1000: 0.6397
2021-11-10 08:42:00 - 

2021-11-10 08:42:00 - MAP@1: 0.4961
2021-11-10 08:42:00 - MAP@3: 0.5428
2021-11-10 08:42:00 - MAP@5: 0.5593
2021-11-10 08:42:00 - MAP@10: 0.5700
2021-11-10 08:42:00 - MAP@100: 0.5749
2021-11-10 08:42:00 - MAP@1000: 0.5751
2021-11-10 08:42:00 - 

2021-11-10 08:42:00 - Recall@1: 0.4961
2021-11-10 08:42:00 - Recall@3: 0.5961
2021-11-10 08:42:00 - Recall@5: 0.6639
2021-11-10 08:42:00 - Recall@10: 0.7249
2021-11-10 08:42:00 - Recall@100: 0.8232
2021-11-10 08:42:00 - Recall@1000: 0.8767
2021-11-10 08:42:00 - 

2021-11-10 08:42:00 - P@1: 0.5100
2021-11-10 08:42:00 - P@3: 0.2100
2021-11-10 08:42:00 - P@5: 0.1427
2021-11-10 08:42:00 - P@10: 0.0797
2021-11-10 08:42:00 - P@100: 0.0092
2021-11-10 08:42:00 - P@1000: 0.0010


In [ ]:
!pip install -U -q kaggle==1.5.8
!mkdir ~/.kaggle
# Upload kaggle.json before running this cell
!mv /content/kaggle.json ~/.kaggle/
# Move all json's to kaggle dataset
!mkdir bm25_results
!mv *.json bm25_results/
!kaggle datasets init -p bm25_results
# > Modify metadata

     |████████████████████████████████| 59 kB 4.9 MB/s 
Data package template written to: bm25_results/dataset-metadata.json


In [ ]:
# Upload to kaggle
!kaggle datasets create -p bm25_results/ --dir-mode zip

Starting upload for file results_cqadupstack_english.json
100% 29.2M/29.2M [00:02<00:00, 10.6MB/s]
Upload successful: results_cqadupstack_english.json (29MB)
Starting upload for file results_webis-touche2020.json
100% 2.51M/2.51M [00:02<00:00, 1.11MB/s]
Upload successful: results_webis-touche2020.json (3MB)
Starting upload for file results_cqadupstack_physics.json
100% 19.3M/19.3M [00:05<00:00, 3.82MB/s]
Upload successful: results_cqadupstack_physics.json (19MB)
Starting upload for file results_cqadupstack_unix.json
100% 20.3M/20.3M [00:02<00:00, 7.93MB/s]
Upload successful: results_cqadupstack_unix.json (20MB)
Starting upload for file results_cqadupstack_tex.json
100% 55.6M/55.6M [00:03<00:00, 17.7MB/s]
Upload successful: results_cqadupstack_tex.json (56MB)
Starting upload for file results_arguana.json
100% 66.3M/66.3M [00:02<00:00, 24.0MB/s]
Upload successful: results_arguana.json (66MB)
Starting upload for file results_fiqa.json
100% 12.6M/12.6M [00:01<00:00, 7.12MB/s]
Upload succes

##### Run on BEIR - Extra datasets (For BioASQ, see separate NB)

In [ ]:
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/

In [ ]:
!kaggle datasets download -d signal1m
!unzip ./signal1m.zip

 89% 137M/154M [00:12<00:01, 13.6MB/s]
100% 154M/154M [00:12<00:00, 12.6MB/s]
Archive:  ./signal1m.zip
  inflating: signal1m/corpus.jsonl   
  inflating: signal1m/qrels/test.tsv  
  inflating: signal1m/queries.jsonl  


In [ ]:
!kaggle datasets download -d trecnews
!unzip ./trecnews.zip

 99% 809M/820M [00:29<00:00, 17.8MB/s]
100% 820M/820M [00:29<00:00, 29.2MB/s]
Archive:  ./trecnews.zip
  inflating: trec-news/corpus.jsonl  
  inflating: trec-news/qrels/test.tsv  
  inflating: trec-news/queries.jsonl  


In [ ]:
!kaggle datasets download -d robust
!unzip ./robust.zip

In [ ]:
import json

from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval


def run_bm25(dataset, data_path, initialize = True, hostname = "localhost"):
    corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
    model = BM25(index_name=dataset, hostname=hostname, initialize=initialize)
    retriever = EvaluateRetrieval(model)
    #### Retrieve dense results (format of results is identical to qrels)
    results = retriever.retrieve(corpus, queries)
    # Save scores for top 1000 docs for each query, i.e. 1000 * queries lines
    with open(f"./results_{dataset}.json", 'w') as fp:
        json.dump(results, fp)
    ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
    return ndcg


### Signal 1M

#run_bm25("signal1m", "signal1m")

##### TREC-News

#run_bm25("trec-news", "trec-news")

##### TREC-Robust

#run_bm25("robust04", "robust04")

/usr/local/lib/python3.7/dist-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/14913938 [00:00<?, ?it/s]

In [ ]:
!mv /content/TREC-Disk-4 /root/.ir_datasets/trec-robust04/trec45/
!mv /content/TREC-Disk-5 /root/.ir_datasets/trec-robust04/trec45/

In [ ]:
!pip install -q ir_datasets
!pip install -q jsonlines

In [ ]:
import ir_datasets
dataset = ir_datasets.load("trec-robust04")
for query in dataset.queries_iter():
    print(query)
    break

[INFO] Please confirm you agree to the TREC Robust 2004 data usage agreement found at <https://trec.nist.gov/data/cd45/index.html>
[INFO] [starting] https://trec.nist.gov/data/robust/04.testset.gz
[INFO] [finished] https://trec.nist.gov/data/robust/04.testset.gz: [00:00] [34.3kB] [195kB/s]
                                                                           

TrecQuery(query_id='301', title='International Organized Crime', description='Identify organizations that participate in international criminal\nactivity, the activity, and, if possible, collaborating organizations\nand the countries involved.', narrative='A relevant document must as a minimum identify the organization and the\ntype of illegal activity (e.g., Columbian cartel exporting cocaine).\nVague references to international drug trade without identification of\nthe organization(s) involved would not be relevant.')


In [ ]:
### Robust04 preprocessing ###

# Also see https://ir-datasets.com/trec-robust04.html#trec-robust04

import jsonlines
import ir_datasets

dataset = ir_datasets.load("trec-robust04")

with jsonlines.open(f'queries.jsonl', 'w') as jsonl_out:
    for query in dataset.queries_iter():
        line = {"_id": query.query_id, "text": query.description}
        jsonl_out.write(line)

with jsonlines.open(f'corpus.jsonl', 'w') as jsonl_out:
    for doc in dataset.docs_iter():
        line = {"_id": doc.doc_id, "text": doc.text}
        jsonl_out.write(line)

with open("test.tsv", "w") as record_file:
    record_file.write("query-id\tcorpus-id\tscore\n")
    for qrel in dataset.qrels_iter():
        record_file.write(f"{qrel.query_id}\t{qrel.doc_id}\t{qrel.relevance}\n")


!mkdir robust04
!mkdir robust04/qrels

!mv /content/queries.jsonl robust04/
!mv /content/corpus.jsonl robust04/
!mv /content/test.tsv robust04/qrels/test.tsv

In [ ]:
def compute_result(dataset, data_path, initialize = True, hostname = "localhost", k_values=[1, 10, 100, 1000]):
    corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
    with open(f"./results_{dataset}.json", 'r') as fp:
        results = json.load(fp)
    ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, results, k_values)
    return ndcg

print(compute_result("signal1m", "signal1m"))
print(compute_result("trec-news", "trec-news"))

2021-11-23 04:20:03 - Loading Corpus...


  0%|          | 0/2866316 [00:00<?, ?it/s]

2021-11-23 04:20:22 - Loaded 2866316 TEST Documents.
2021-11-23 04:20:22 - Doc Example: {'text': 'This Boston college professor who lives in #NH is on leave after being arrested for child pornography, endangerment:', 'title': ''}
2021-11-23 04:20:22 - Loading Queries...
2021-11-23 04:20:22 - Loaded 97 TEST Queries.
2021-11-23 04:20:22 - Query Example: VIDEO:Good Samaritans Stop Alleged Hit-and-Run Driver in Miami
2021-11-23 04:20:22 - 

2021-11-23 04:20:22 - NDCG@1: 0.4021
2021-11-23 04:20:22 - NDCG@10: 0.3302
2021-11-23 04:20:22 - NDCG@100: 0.3313
2021-11-23 04:20:22 - NDCG@1000: 0.3890
2021-11-23 04:20:22 - 

2021-11-23 04:20:22 - MAP@1: 0.0293
2021-11-23 04:20:22 - MAP@10: 0.1249
2021-11-23 04:20:22 - MAP@100: 0.1765
2021-11-23 04:20:22 - MAP@1000: 0.1839
2021-11-23 04:20:22 - 

2021-11-23 04:20:22 - Recall@1: 0.0293
2021-11-23 04:20:22 - Recall@10: 0.1628
2021-11-23 04:20:22 - Recall@100: 0.3737
2021-11-23 04:20:22 - Recall@1000: 0.5608
2021-11-23 04:20:22 - 

2021-11-23 04:20:22 -

  0%|          | 0/594977 [00:00<?, ?it/s]

2021-11-23 04:20:45 - Loaded 594977 TEST Documents.
2021-11-23 04:20:45 - Doc Example: {'text': 'NEW ORLEANS — Whenever a Virginia Tech offensive coach is asked how the most prolific receiving duo in school history came to be, inevitably the first road game in 2008 against North Carolina comes up. Midway through the first quarter, Virginia Tech had to call two timeouts in a row because then-freshmen Jarrett Boykin and Danny Coale couldn’t seem to line up right, and “they had those big eyes out there looking around,” Kevin Sherman, their position coach, said recently. Now that Boykin and Coale have only Tuesday’s Sugar Bowl remaining before leaving Virginia Tech with every major school record for a wide receiver, they’ve taken a different stance. “I still don’t think that was on us. Macho [Harris] was in the game and he lined up wrong,” said Boykin, as Coale sat next to him nodding in agreement. Just add that to the list of slights these seniors have had to overcome. Boykin has been the

In [ ]:
from google.colab import files
files.download('/content/results_signal1m.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##### Evaluate BEIR

##### References

- https://stackoverflow.com/questions/58418844/google-colaboratory-can-i-use-it-to-connect-to-a-local-server
